<a href="https://colab.research.google.com/github/louiezzang/next-gpt/blob/main/examples/chatgpt_replica_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# chatGPT replica


What is RLHF? <br>
See [this link](https://gist.github.com/JoaoLages/c6f2dfd13d2484aa8bb0b2d567fbf093).

<br>

**Example of RLHF dataset**:

Total 3 datasets are needed for training the 3 steps(SFT, RM and PPO)
- [Example of dataset](https://github.com/nebuly-ai/nebullvm/tree/main/apps/accelerate/chatllama#dataset-preparation)
- [Example of dataset 1](https://huggingface.co/datasets/stanfordnlp/SHP)
- [Example of dataset 2](https://huggingface.co/datasets/Anthropic/hh-rlhf)

step1) Dataset for SFT(Supervised Fine-tuning training)
```json
[
    {
        "prompt": "",
        "completion": ""        
    }, ...
]
```

step2) Dataset for RM(Reward Model) training: There are multiple completetions with human rated ranking score for one prompt.
```json
[
    {
        "prompt": "",
        "completion_1": "",
        "completion_2": "",
        "completion_3": "",            
        "ranking": [1, 0, 2]
    }, ...
]
```
    
step3) Dataset for PPO(RLHF) training: It only consists of prompt.
```json
[
    {
        "prompt": ""
    }, ...
]
```

# Colab environment setup

#### Installation (python>=3.8)

In [1]:
# Install next-gpt lib.
!rm -rf ./next-gpt/
!git clone https://github.com/louiezzang/next-gpt.git
%cd next-gpt/
!pip install .
%cd ../

Cloning into 'next-gpt'...
remote: Enumerating objects: 564, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 564 (delta 104), reused 147 (delta 68), pack-reused 348
Receiving objects: 100% (564/564), 195.64 KiB | 7.83 MiB/s, done.
Resolving deltas: 100% (272/272), done.
/content/next-gpt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/next-gpt
  Preparing metadata (setup.py) ... done
  Created wheel for next-gpt: filename=next_gpt-1.0.0-py3-none-any.whl size=58894 sha256=d809070da201811a3213cffde44e0c6227b4eca5aaf39055f8eedfbd35b617e0
  Stored in directory: /root/.cache/pip/wheels/f4/0d/a2/307ec9a6214260bfd63facee827d7bbef5c1ba9618277ea1b5
Successfully built next-gpt
  Attempting uninstall: next-gpt
    Found existing installation: next-gpt 1.0.0
    Uninstalling next-gpt-1.0.0:
      Successfully uninstalled next-gpt-1.0.0
/content


# Step 1) SFT: Surpervised Fine-tuning
Build a Supervised Fine-tuning model to answer well to the question.

- Refereneces
  - [fine tuning code_1](https://github.com/philschmid/fine-tune-GPT-2/blob/master/Fine_tune_a_non_English_GPT_2_Model_with_Huggingface.ipynb)
  - [fine tuning code_2](https://github.com/Beomi/KoAlpaca/blob/main/train.py)


- SFT(Supervised Fine Tuning)
- Fine-tune a pretrained LLM on a specific domain or corpus of instructions and human demonstrations

- Dataset example
```json
[
    {
        "prompt": "",
        "completion": ""        
    }, ...
]
```

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import json
import yaml
import argparse

import numpy as np
import pandas as pd

import torch
from datasets import load_dataset
import transformers

from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForCausalLM, pipeline, 
    TrainingArguments, AutoModelWithLMHead,
    ProgressCallback
)
from nextgpt.finetuning import (
    SupervisedDataset, DataCollatorForSupervisedDataset,
    SupervisedTrainer, LoggingCallback
)

PT_MODEL_NAME = "gpt2"

In [3]:
# Define arguments.
parser = argparse.ArgumentParser()
parser.add_argument("--model_name", type=str, default="gpt2", choices=["gpt2'", "bloom", "opt"])
parser.add_argument("--max_epochs", type=int, default=1)
parser.add_argument("--train_batch_size", type=int, default=4)
parser.add_argument("--output_dir", type=str, default="./output_1_sft")

args = parser.parse_args(args=[])
print(args)

Namespace(model_name='gpt2', max_epochs=1, train_batch_size=4, output_dir='./output_1_sft')


In [4]:
# Get the tokenizer.
tokenizer = transformers.AutoTokenizer.from_pretrained(args.model_name, 
                                          bos_token="<|startoftext|>",
                                          eos_token="<|endoftext|>", 
                                          pad_token="<|pad|>")
print(tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|pad|>'})


In [5]:
dataset_webgpt_comp = load_dataset("openai/webgpt_comparisons", split="train[:20%]")

In [6]:
data_list = []
for row in dataset_webgpt_comp:
  question = row["question"]["full_text"]
  answer_0 = row["answer_0"]
  data_list.append({
      "prompt": question,
      "completion": answer_0
  })

In [7]:
PROMPT_TEMPLATE = (
  "Below is an instruction that describes a task, paired with an input that provides further context.\n\n"
  "Write a response that appropriately completes the request.\n\n"
  "### Instruction:\n{prompt}\n\n### Response:"
)

In [8]:
dataset = SupervisedDataset(
    data=data_list,
    tokenizer=tokenizer, 
    prompt_template=PROMPT_TEMPLATE,
    completion_field="completion",
    verbose=True)

# Split train and val dataset.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Data collator.
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

100%|██████████| 3916/3916 [00:00<00:00, 221622.60it/s]


Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
Voiced by Harry Shearer, what Simpsons character was modeled after Ted Koppel?

### Response:
The Simpsons character that was possibly based on Ted Koppel is Kent Brockman.  He is a local news anchor in Springfield and is modeled after Ted Koppel. [1]<|endoftext|>
Tokenizing inputs... This may take some time...
Loading data done!!: 3916


In [9]:
# Load the pretrained model.
model = AutoModelForCausalLM.from_pretrained(args.model_name)
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 768)

In [ ]:
# Train arguments.
training_args = TrainingArguments(
    output_dir="./train_1_sft", # the output directory
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=args.max_epochs, # number of training epochs
    per_device_train_batch_size=args.train_batch_size, # batch size for training
    per_device_eval_batch_size=4, # batch size for evaluation
    eval_steps=3, # number of update steps between two evaluations.
    save_steps=100, # after # steps model is saved 
    warmup_steps=5, # number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)

# Train the model.
trainer = SupervisedTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=None,
    # callbacks=[ProgressCallback, LoggingCallback(logger=None)],
)

trainer.train()
trainer.save_state()
trainer.safe_save_model(output_dir=args.output_dir)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [ ]:
# Inference test.
generator = pipeline("text-generation", model=args.output_dir, tokenizer=tokenizer)

generation_args = dict(
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    # bos_token="<|startoftext|>",
    # eos_token="<|endoftext|>", 
    # pad_token="<|pad|>",
    max_new_tokens=64,
    do_sample=True,
    top_k=30,
    top_p=0.95,
    temperature=1.9, 
    #max_length=300, 
    #num_return_sequences=20
    early_stopping=True,
)

In [ ]:
test_list = data_list[-5:]

test_prompt_list = []
actual_completion_list = []
for row in test_list:
    text_input = row
    prompt = PROMPT_TEMPLATE.format_map(text_input)
    test_prompt_list.append(prompt)
    actual_completion_list.append(text_input["completion"])

result_list = generator(test_prompt_list, **generation_args)
for prompt, result, actual_response in zip(test_prompt_list, result_list, actual_completion_list):
    print("")
    print("-" * 70)
    print(("completion: %s" % (result[0]["generated_text"])))
    print(f"\n### Actual answer:\n{actual_response}")

# Step 2) RM: Reward Model
Train Reward Model to generate the better answer by giving a reward to the better answer.
- Dataset example
```json
[
    {
        "prompt": "",
        "completion_1": "",
        "completion_2": "",
        "completion_3": "",            
        "ranking": [1, 0, 2]
    }, ...
]
```
- Dataset sources
  - [Dahoas/rm-static](https://huggingface.co/datasets/Dahoas/rm-static)
  - [openai/webgpt_comparisons](https://huggingface.co/datasets/openai/webgpt_comparisons)
  - [openai/summarize_from_feedback](https://huggingface.co/datasets/openai/summarize_from_feedback)
  - [Dahoas/instruct-synthetic-prompt-responses](https://huggingface.co/datasets/Dahoas/synthetic-instruct-gptj-pairwise)

- References
    - [train_reward_model.py](https://github.com/hpcaitech/ColossalAI/blob/main/applications/Chat/examples/train_reward_model.py)
    - [train_prompts.py](https://github.com/hpcaitech/ColossalAI/blob/main/applications/Chat/examples/train_prompts.py)

In [ ]:
import os
import json
import argparse

import torch
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, BloomTokenizerFast
from transformers.models.gpt2.tokenization_gpt2 import GPT2Tokenizer
import loralib as lora

from nextgpt.rlhf.dataset import RewardDataset
from nextgpt.rlhf.models.base import RewardModel
from nextgpt.rlhf.models.bloom import BLOOMRM
from nextgpt.rlhf.models.gpt import GPTRM
from nextgpt.rlhf.models.opt import OPTRM
from nextgpt.rlhf.models import LogExpLoss, LogSigLoss
from nextgpt.rlhf.trainer import RewardModelTrainer
from nextgpt.rlhf.trainer.strategies import DDPStrategy, NaiveStrategy

In [ ]:
# Define arguments.
parser = argparse.ArgumentParser()
parser.add_argument("--output_dir", type=str, default="/output_2_rm")
parser.add_argument("--strategy",
                    choices=["naive", "ddp"],
                    default="naive")
parser.add_argument("--model_name", type=str, default="gpt2", choices=["gpt2", "bloom", "opt"])
parser.add_argument("--pretrain", type=str, default="gpt2")
parser.add_argument("--model_path", type=str, default=None)
parser.add_argument('--need_optim_ckpt', type=bool, default=False)
parser.add_argument("--max_epochs", type=int, default=10)
parser.add_argument("--batch_size", type=int, default=4)
parser.add_argument("--lora_rank", type=int, default=0, help="low-rank adaptation matrices rank")
parser.add_argument("--loss_fn", type=str, default="log_sig", choices=["log_sig", "log_exp"])
parser.add_argument("--max_len", type=int, default=512)

args = parser.parse_args(args=[])

# For test.
args.max_epochs = 3
args.pretrain = "gpt2" # pretrained initial model.
args.max_len = 1024
args.verbose = True

print(args)
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

In [ ]:
# Configure strategy.
if args.strategy == "naive":
    strategy = NaiveStrategy()
elif args.strategy == "ddp":
    strategy = DDPStrategy()
else:
    raise ValueError(f"Unsupported strategy: {args.strategy}")

In [ ]:
# Configure model, tokenizer.
with strategy.model_init_context():
    # Load pretrained gpt2.
    if args.model_name == "gpt2":
        # Get the tokenizer.
        tokenizer = transformers.AutoTokenizer.from_pretrained(
            args.model_name, 
            bos_token="<|startoftext|>",
            eos_token="<|endoftext|>", 
            pad_token="<|pad|>",
            padding_side="right", 
            model_max_length=args.max_len,
            )
        print(tokenizer)
        model = GPTRM(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        model.resize_token_embeddings(len(tokenizer)) 
    elif args.model == "bloom":
        model = BLOOMRM(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        tokenizer = BloomTokenizerFast.from_pretrained(args.pretrain)
    elif args.model == "opt":
        model = OPTRM(pretrained=args.pretrain, lora_rank=args.lora_rank).to(torch.cuda.current_device())
        tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")      
    else:
        raise ValueError(f"Unsupported model: {args.model_name}")

    # Load the supervised finetuning model state dict if it is specified.
    # However, we will train the reward model from the initial language model instead of supervised finetuning model.
    if args.model_path is not None:
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        state_dict = torch.load(args.model_path)
        model.model.load_state_dict(state_dict)

    # This float16 or `model.half()` might cause loss NaN issue!!!
    # See:
    #   https://stackoverflow.com/questions/65332165/loss-is-nan-when-fine-tuning-huggingface-nli-model-both-roberta-bart
    #   https://github.com/huggingface/transformers/issues/9160
    model = model.to(torch.float16)

In [ ]:
# Get the dataset.
dataset_webgpt_comp = load_dataset("openai/webgpt_comparisons", split="train[:20%]")

In [ ]:
# Convert data into ranking format.
data_list_ranking = []
for row in dataset_webgpt_comp:
    question = row["question"]["full_text"]
    answer_0 = row["answer_0"]
    answer_1 = row["answer_1"]
    score_0 = row["score_0"]
    score_1 = row["score_1"]
    if answer_0 == "" or answer_1 == "" or (score_0 == score_1):
        continue

    ranking = [0 if score_0 > score_1 else 1, 0 if score_0 < score_1 else 1]
    data_list_ranking.append({
        "prompt": PROMPT_TEMPLATE.format_map({"prompt": question}),
        "completion_0": answer_0,
        "completion_1": answer_1,
        "ranking": ranking
    })

data_list_ranking[:2]

In [ ]:
# Make ranking data to chosen, rejetced data for reward model dataset.
total_data_ranking2chosen = []
for tmp in data_list_ranking:
    one_data_ranking2chosen = []

    # data 1) 0 VS 1
    data = {}
    data["prompt"] = tmp["prompt"]
    if tmp["ranking"][0] < tmp["ranking"][1]:
        data["chosen"] = tmp["completion_0"]
        data["rejected"] = tmp["completion_1"]
    else:
        data["chosen"] = tmp["completion_1"]
        data["rejected"] = tmp["completion_0"]
    one_data_ranking2chosen.append(data)

    # # data 2) 0 VS 2
    # data = {}
    # data["prompt"] = tmp["prompt"]
    # if tmp["ranking"][0] < tmp["ranking"][2]:
    #     data["chosen"] = tmp["completion_0"]
    #     data["rejected"] = tmp["completion_2"]
    # else:
    #     data["chosen"] = tmp["completion_2"]
    #     data["rejected"] = tmp["completion_0"]
    # one_data_ranking2chosen.append(data)

    # # data 1) 1 VS 2
    # data = {}
    # data["prompt"] = tmp["prompt"]
    # if tmp["ranking"][1] < tmp["ranking"][2]:
    #     data["chosen"] = tmp["completion_1"]
    #     data["rejected"] = tmp["completion_2"]
    # else:
    #     data["chosen"] = tmp["completion_2"]
    #     data["rejected"] = tmp["completion_1"]
    # one_data_ranking2chosen.append(data)


    total_data_ranking2chosen.extend(one_data_ranking2chosen)


print("before data num: %d" % (len(data_list_ranking)))
print("after data num: %d" % (len(total_data_ranking2chosen)))
print("data example: \n%s" % total_data_ranking2chosen[1])

In [ ]:
# Prepare for data and dataset.
import random
random.seed(230319)

random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[1])

# train_data = total_data_ranking2chosen[:-1000]
# eval_data = total_data_ranking2chosen[-1000:0]
# We just select very small set of data for a quicker training.
train_data = total_data_ranking2chosen[:100]
val_data = total_data_ranking2chosen[100:130]
eval_data = total_data_ranking2chosen[130:160]

train_dataset = RewardDataset(train_data, tokenizer, args.max_len)
val_dataset = RewardDataset(val_data, tokenizer, args.max_len)
eval_dataset = RewardDataset(eval_data, tokenizer, args.max_len)

# Check
idx = 10
print("#" * 70)
print("## prompt ##")
print(train_data[idx]["prompt"])
print("#" * 70)
print("## chosen ##")
print(train_data[idx]["chosen"])
print("#" * 70)
print("## rejected ##")
print(train_data[idx]["rejected"])

In [ ]:
# Configure optimizer.
optim = Adam(model.parameters(), lr=5e-5)

# Configure loss function.
if args.loss_fn == "log_sig":
    loss_fn = LogSigLoss()
elif args.loss_fn == "log_exp":
    loss_fn = LogExpLoss()
else:
    raise ValueError(f"Unsupported loss function: {args.loss_fn}")

In [ ]:
trainer = RewardModelTrainer(model=model,
                            strategy=strategy,
                            optim=optim,
                            loss_fn=loss_fn,
                            train_dataset=train_dataset,
                            valid_dataset=val_dataset,
                            eval_dataset=eval_dataset,
                            batch_size=args.batch_size,
                            max_epochs=args.max_epochs)

In [ ]:
# Train!!!
trainer.fit()

# Save model checkpoint after fitting on only rank0.
# strategy.save_model(model, os.path.join(args.output_dir, "rm.pt"), only_rank0=True)
trainer.save_model(path=os.path.join(args.output_dir, "rm.pt"), only_rank0=True)

# Save optimizer checkpoint on all ranks.
if args.need_optim_ckpt:
    strategy.save_optimizer(trainer.optimizer,
                            os.path.join(args.output_dir, "rm_optim_checkpoint_%d.pt" % (torch.cuda.current_device())),
                            only_rank0=False)

# Step 3) PPO: Proximal Policy Optimization
Further fine-tune the LLM from step 1 with the reward model and this dataset using RL (eg. PPO).

- References
    - [train_prompts.py](https://github.com/hpcaitech/ColossalAI/blob/main/applications/Chat/examples/train_prompts.py)